# DomoAuth

> Fill in a module description here


In [ ]:
# | default_exp client.DomoAuth

In [ ]:
# | hide
from nbdev.showdoc import *
import fastcore.test as fctest

In [ ]:
# | exporti
from dataclasses import dataclass, field
from typing import Optional, Union

import httpx

import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoError as de

# Authentication Routes

Stand alone functions for users who prefer a functional programming approach

## Full Auth Route - username and password authentication


In [ ]:
# | export
async def get_full_auth(
    domo_instance: str,  # domo_instance.domo.com
    domo_username: str,  # email address
    domo_password: str,
    session: Optional[httpx.AsyncClient] = None,
    debug_api: bool = False
) -> rgd.ResponseGetData:
    """uses username and password authentication to retrieve a full_auth access token"""

    is_close_session = False

    if not session:
        is_close_session = True
        session = httpx.AsyncClient()

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    tokenHeaders = {"Content-Type": "application/json"}

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    if debug_api:
        print(body, url)

    res = await session.request(method="POST", url=url, headers=tokenHeaders, json=body)

    if is_close_session:
        await session.aclose()

    return rgd.ResponseGetData._from_httpx_response(res)

#### Sample Implementations of get_full_auth


In [ ]:
import os

res = await get_full_auth(
    domo_instance="domo-community",
    domo_username="test@test.com",
    domo_password="fake password",
)
res

ResponseGetData(status=200, response={'success': False, 'reason': 'INVALID_CREDENTIALS'}, is_success=True)

In [ ]:
res = await get_full_auth(domo_instance="test",
                          domo_username="fake@test.com", 
                          domo_password="fake password")


assert res.status == 403


In [ ]:
res = await get_full_auth(domo_instance="domo-community",
                          domo_username="jae@onyxreporting.com",
                          domo_password=os.environ['DOJO_PASSWORD'])
assert res.status == 200


The 403 response is expected because test.domo.com presumeably does not exist or access if forbidden.


## Developer Auth Route - client_id and secret authentication


In [ ]:
# | export
async def get_developer_auth(
    domo_client_id: str,
    domo_client_secret: str,
    session: Optional[httpx.AsyncClient] = None,
    debug_api: bool = False
) -> rgd.ResponseGetData:

    """
    only use for authenticating against apis documented under developer.domo.com
    """
    is_close_session = False

    if not session:
        is_close_session = True
        session = httpx.AsyncClient(
            auth=httpx.BasicAuth(domo_client_id, domo_client_secret)
        )

    url = "https://api.domo.com/oauth/token?grant_type=client_credentials"

    if debug_api:
        print(url, domo_client_id, domo_client_secret)

    res = await session.request(method="GET", url=url)

    if is_close_session:
        await session.aclose()

    return rgd.ResponseGetData._from_httpx_response(res)

#### Sample Implementations of get_developer_auth

The 401 response is expected because we are using invalid credentials


In [ ]:
await get_developer_auth(domo_client_id="test_id",
                         domo_client_secret="test_secret"
                         )


ResponseGetData(status=401, response='Unauthorized', is_success=False)

## Access Token Auth Route - access_token authentication

This access_token based authentication assumes the user has been provided a valid access token from Domo > Admin > Authentication > Access Token so authentication routes are actually not required.

Per the Domo JavaCLI implementation, users can test the validity of the access_token agains the 'me' API

!! Note about the Me API !!
It appears that access_token authentication will direct the


In [ ]:
# | export
async def test_access_token(
    domo_access_token: str,  # as provided in Domo > Admin > Authentication > AccessTokens
    domo_instance: str,  # <domo_instance>.domo.com
    session: Optional[httpx.AsyncClient] = None,
    debug_api: bool = False
):
    """
    will attempt to validate against the 'me' API.
    This is the same authentication test the Domo Java CLI uses.
    """

    is_close_session = False

    if not session:
        is_close_session = True
        session = httpx.AsyncClient()

    url = f"https://{domo_instance}.domo.com/api/content/v2/users/me"

    tokenHeaders = {"X-DOMO-Developer-Token": domo_access_token}

    if debug_api:
        print(url,tokenHeaders)

    res = await session.request(method="GET", headers=tokenHeaders, url=url)

    if is_close_session:
        await session.aclose()

    return rgd.ResponseGetData._from_httpx_response(res)

#### Sample implementation of `test_access_token`


In [ ]:
import os

await test_access_token(
    domo_instance="domo-community",
    domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN']
)

ResponseGetData(status=200, response={'id': 1893952720, 'invitorUserId': 587894148, 'displayName': 'Jae Wilson1', 'department': 'Business Improvement', 'userName': 'jae@onyxreporting.com', 'emailAddress': 'jae@onyxreporting.com', 'avatarKey': 'c605f478-0cd2-4451-9fd4-d82090b71e66', 'accepted': True, 'userType': 'USER', 'modified': 1679004195646, 'created': 1588960518, 'role': 'Admin', 'rights': 63, 'active': True, 'pending': False, 'systemUser': False, 'anonymous': False}, is_success=True)

# Classes

Different Domo Auth classes will have a variety of required vs optional parameters. To avoid multiple initialization and post_intialization statements, we mix multiple classes together such that classes with optional parameters are mixed in before classes with required parameters.


In [ ]:
# | exporti
@dataclass
class _DomoAuth_Required:
    """required parameters for all Domo Auth classes"""

    domo_instance: str

    def __post_init__(self):
        if self.domo_instance:
            self.set_manual_login()

    def set_manual_login(self):
        self.url_manual_login = (
            f"https://{self.domo_instance}.domo.com/auth/index?domoManualLogin=true"
        )


@dataclass
class _DomoAuth_Optional:
    """parameters are defined after initialization"""

    token: Optional[str] = field(default=None, repr=False)
    token_name: Optional[str] = field(default=None)
    user_id: Optional[str] = field(default=None, repr=False)
    auth_header: dict = field(default_factory=dict, repr=False)

    is_valid_token: bool = None

    url_manual_login: Optional[str] = None

    async def get_auth_token(self) -> Union[str, None]:
        """placeholder method"""
        pass

    async def generate_auth_header(self) -> Union[dict, None]:
        """returns auth header appropriate for this authentication method"""
        pass

    async def print_is_token(self, token_name=None) -> None:
        self.token_name = token_name or self.token_name

        if not self.token:
            await self.get_auth_token()

        token_str = f"{self.token_name} "

        if not self.token:
            print(
                f"🚧 failed to retrieve {token_str if token_name else ''}token from {self.domo_instance}")
            return False

        print(
            f"🎉 {token_str if token_name else ''}token retrieved from {self.domo_instance} ⚙️")
        return True


In [ ]:
# | export
@dataclass
class DomoAuth(_DomoAuth_Optional, _DomoAuth_Required):
    """abstract DomoAuth class"""

In [ ]:
# Attributes of `DomoAuth`
domo_instance = "test"
[attr for attr in dir(DomoAuth(domo_instance)) if not attr.startswith("__")]

['auth_header',
 'domo_instance',
 'generate_auth_header',
 'get_auth_token',
 'is_valid_token',
 'print_is_token',
 'set_manual_login',
 'token',
 'token_name',
 'url_manual_login',
 'user_id']

In [ ]:
# validate can print manual login link
domo_instance = "test"

da = DomoAuth(domo_instance)
da.url_manual_login

'https://test.domo.com/auth/index?domoManualLogin=true'

## DomoAuth Error Classes


In [ ]:
# | export
class InvalidCredentialsError(de.DomoError):
    """return invalid credentials sent to API"""

    def __init__(
        self,
        function_name: Optional[str] = None,
        status: Optional[int] = None,  # API request status
        message="invalid credentials",
        domo_instance: Optional[str] = None,
    ):

        super().__init__(status=status, message=message, domo_instance=domo_instance)


class InvalidAuthTypeError(de.DomoError):
    """return invalid Auth type sent to API"""

    def __init__(
        self,
        required_auth_type :DomoAuth ,
        function_name: Optional[str] = None,
        domo_instance: Optional[str] = None,
    ):
        message = f"This API rquires {required_auth_type.__name__}"

        super().__init__(message=message, domo_instance=domo_instance, function_name = function_name)

class InvalidInstanceError(de.DomoError):
    """return if invalid domo_instance sent to API"""

    def __init__(
        self,
        function_name: Optional[str] = None,
        status: Optional[int] = None,
        message="invalid instance",
        domo_instance: Optional[str] = None,
    ):
        super().__init__(status=status, message=message, domo_instance=domo_instance)
    

class NoAccessTokenReturned(de.DomoError):
    def __init__(
        self,
        function_name: Optional[str] = None,
        status: Optional[int] = None,
        message :str = "No AccessToken returned",
        domo_instance: Optional[str] = None,
    ):
        super().__init__(status = status, message = message, domo_instance = domo_instance)

In [ ]:
# | hide
# validates that Errors can compile
fctest.test_eq(isinstance(InvalidCredentialsError(), InvalidCredentialsError), True)
fctest.test_eq(isinstance(InvalidCredentialsError(), InvalidCredentialsError), True)
fctest.test_eq(isinstance(InvalidInstanceError(), InvalidInstanceError), True)
fctest.test_eq(isinstance(NoAccessTokenReturned(), NoAccessTokenReturned), True)


## DomoFullAuth


In [ ]:
# | exporti
@dataclass
class _DomoFullAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_username: str
    domo_password: str = field(repr=False)


In [ ]:
# | export
@dataclass
class DomoFullAuth(_DomoAuth_Optional, _DomoFullAuth_Required):
    """use for full authentication token"""

    def generate_auth_header(self, token: str) -> dict:
        self.auth_header = {"x-domo-authentication": token}
        return self.auth_header

    async def get_auth_token(
        self,
        session: Optional[httpx.AsyncClient] = None,
        debug_api : bool = False
    ) -> str:
        """returns `token` if valid credentials provided else raises Exception and returns None"""

        res = await get_full_auth(
            domo_instance=self.domo_instance,
            domo_username=self.domo_username,
            domo_password=self.domo_password,
            session=session,
            debug_api = debug_api
        )

        if res.is_success and res.response.get("reason") == "INVALID_CREDENTIALS":
            self.is_valid_token = False
            raise InvalidCredentialsError(
                function_name = "get_auth_token",
                status=res.status,
                message=str(res.response.get("reason")),
                domo_instance=self.domo_instance,
            )

        if res.status == 403:
            self.is_valid_token = False
            raise InvalidInstanceError(
                function_name = "get_auth_token",
                status=res.status,
                message="INVALID INSTANCE",
                domo_instance=self.domo_instance,
            )
        
        if res.is_success and res.response == {}:
            self.is_valid_token = False
            raise NoAccessTokenReturned(
                function_name="get_auth_token",
                status=res.status, 
                domo_instance=self.domo_instance)
        
        self.is_valid_token = True

        token = str(res.response.get("sessionToken"))
        self.token = token
        self.user_id = str(res.response.get("userId"))

        self.auth_header = self.generate_auth_header(token=token)

        if not self.token_name:
            self.token_name = "full_auth"

        return self.token

#### sample implementations of DomoFullAuth


In [ ]:
try:
    full_auth = DomoFullAuth(domo_instance="domo-community",
                             domo_username="test12@domo.com",
                             domo_password="test1234")
    res = await full_auth.get_auth_token()

except InvalidCredentialsError as e:
    print(e)

Status 200 - INVALID_CREDENTIALS at domo-community


In [ ]:
try:
    full_auth = DomoFullAuth(domo_instance="test",
                             domo_username="test12@domo.com",
                             domo_password="test1234")
    
    await full_auth.get_auth_token()


except InvalidInstanceError as e:
    print(e)

Status 403 - INVALID INSTANCE at test


In [ ]:
import os
try:
    full_auth = DomoFullAuth(domo_instance="domo-community",
                             domo_username="jae@onyxreporting.com",
                             domo_password=os.environ['DOJO_PASSWORD'])

    await full_auth.get_auth_token()


except InvalidInstanceError as e:
    print(e)

assert(full_auth.token)
full_auth


DomoFullAuth(domo_instance='domo-community', domo_username='jae@onyxreporting.com', token_name='full_auth', is_valid_token=True, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true')

In [ ]:
# | hide
import os

domo_instance = "domo-community"
fctest.test_eq(isinstance(DomoAuth(domo_instance=domo_instance), DomoAuth), True)

fctest.test_eq(
    isinstance(
        DomoFullAuth(
            domo_instance=domo_instance,
            domo_password=os.environ['DOJO_PASSWORD'],
            domo_username='jae@onyxreporting.com',
        ),
        DomoFullAuth,
    ),
    True,
)

## DomoTokenAuth


In [ ]:
# | exporti
@dataclass
class _DomoTokenAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_access_token: str = field(repr=False)

In [ ]:
# | export
@dataclass
class DomoTokenAuth(_DomoAuth_Optional, _DomoTokenAuth_Required):
    

    """
    use for access_token authentication.
    Tokens are generated in domo > admin > access token
    Necessary in cases where direct sign on is not permitted
    """

    def generate_auth_header(self, token: str) -> dict:
        self.auth_header = {"x-domo-developer-token": token}
        return self.auth_header

    async def get_auth_token(
        self, session: Optional[httpx.AsyncClient] = None,
        debug_api : bool = False
    ) -> str:
        """
        updates internal attributes
        having an access_token assumes pre-authenticaiton
        """

        res = await test_access_token(
            domo_instance=self.domo_instance,
            domo_access_token=self.domo_access_token,
            session=session,
            debug_api = debug_api
        )

        if res.status == 401 and res.response == "Unauthorized":
            self.is_valid_token = False
            raise InvalidCredentialsError(
                status=res.status,
                message=res.response,
                domo_instance=self.domo_instance,
            )

        if res.status == 200:
            self.is_valid_token = True

        self.token = self.domo_access_token
        self.user_id = res.response.get("id")

        self.auth_header = self.generate_auth_header(token=self.token)

        if not self.token_name:
            self.token_name = "token_auth"

        return self.token

#### Sample implementation of DomoTokenAuth


In [ ]:
try:
    domo_auth = DomoTokenAuth(domo_instance="domo-community",
                              domo_access_token='fake password')
    token = await domo_auth.get_auth_token()

except InvalidCredentialsError as e:
    print(e)

Status 401 - Unauthorized at domo-community


## DomoDeveloperAuth


In [ ]:
# | exporti
@dataclass
class _DomoDeveloperAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_client_id: str
    domo_client_secret: str = field(repr=False)

In [ ]:
# | export
@dataclass(init=False)
class DomoDeveloperAuth(_DomoAuth_Optional, _DomoDeveloperAuth_Required):
    """use for full authentication token"""

    def __init__(self, domo_client_id: str, domo_client_secret: str):
        self.domo_client_id = domo_client_id
        self.domo_client_secret = domo_client_secret
        self.domo_instance = ""

    def generate_auth_header(self, token: str) -> dict:
        self.auth_header = {"Authorization": "bearer " + token}
        return self.auth_header

    async def get_auth_token(
        self,
        session: Optional[httpx.AsyncClient] = None,
        debug_api : bool = False
    ) -> str:

        res = await get_developer_auth(
            domo_client_id=self.domo_client_id,
            domo_client_secret=self.domo_client_secret,
            session=session,
            debug_api = debug_api
        )

        if res.status == 401:
            self.is_valid_token = False
            raise InvalidCredentialsError(
                status=res.status,
                message=str(res.response),
                domo_instance=self.domo_instance,
            )

        self.is_valid_token = True

        token = str(res.response.get("access_token"))
        self.token = token
        self.user_id = res.response.get("userId")
        self.domo_instance = res.response.get("domain")
        self.set_manual_login()

        self.auth_header = self.generate_auth_header(token=token)

        if not self.token_name:
            self.token_name = "developer_auth"

        return token

#### Sample implementations of DomoDeveloperAuth


In [ ]:
domo_client_id = "test_client"
domo_client_secret = "test_secret"

try:
    domo_auth = DomoDeveloperAuth(domo_client_id, domo_client_secret)
    await domo_auth.get_auth_token()
except InvalidCredentialsError as e:
    print(e)

Status 401 - Unauthorized


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()